# Final Project Cleaning with Pandas

In [3]:
#import sys
#sys.path.append("/opt/anaconda3/lib/python3.8/site-packages")

In [4]:
import pyspark
import pandas as pd
import pyspark.sql
from pyspark.sql import SparkSession

In [5]:
data_2010 = pd.read_csv('circuit_data/circuit_criminal_2010_anon_00.csv')
data_2011 = pd.read_csv('circuit_data/circuit_criminal_2011_anon_00.csv')
data_2012 = pd.read_csv('circuit_data/circuit_criminal_2012_anon_00.csv')
data_2013 = pd.read_csv('circuit_data/circuit_criminal_2013_anon_00.csv')
data_2014 = pd.read_csv('circuit_data/circuit_criminal_2014_anon_00.csv')
data_2015 = pd.read_csv('circuit_data/circuit_criminal_2015_anon_00.csv')
data_2016 = pd.read_csv('circuit_data/circuit_criminal_2016_anon_00.csv')
data_2017 = pd.read_csv('circuit_data/circuit_criminal_2017_anon_00.csv')
data_2018 = pd.read_csv('circuit_data/circuit_criminal_2018_anon_00.csv')
data_2019 = pd.read_csv('circuit_data/circuit_criminal_2019_anon_00.csv')

/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,27,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
data_2010.head(2)

,HearingDate,HearingResult,HearingJury,HearingPlea,HearingType,HearingRoom,fips,Filed,Commencedby,Locality,...,DrivingRestrictions,RestrictionEffectiveDate,RestrictionEndDate,VAAlcoholSafetyAction,RestitutionPaid,RestitutionAmount,Military,TrafficFatality,AppealedDate,person_id
0,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710
1,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710


In [7]:
def unique_arrests(file):
    return file['person_id'].nunique()

In [8]:
# these are for the total population estimates
arrests_2010 = unique_arrests(data_2010)
arrests_2011 = unique_arrests(data_2011)
arrests_2012 = unique_arrests(data_2012)
arrests_2013 = unique_arrests(data_2013)
arrests_2014 = unique_arrests(data_2014)
arrests_2015 = unique_arrests(data_2015)
arrests_2016 = unique_arrests(data_2016)
arrests_2017 = unique_arrests(data_2017)
arrests_2018 = unique_arrests(data_2018)
arrests_2019 = unique_arrests(data_2019)

# How much race data is missing? 
- I'll put this in the corner of my visual!

In [7]:
#data_2010.Race.unique()

array(['Black (Non-Hispanic)', 'White Caucasian (Non-Hispanic)',
       'Asian Or Pacific Islander', 'Hispanic', nan,
       'Other (Includes Not Applicable, Unknown)', 'American Indian'],
      dtype=object)

In [8]:
#data_2010['Race'].value_counts()

White Caucasian (Non-Hispanic)              91889
Black (Non-Hispanic)                        73240
Hispanic                                     1540
Other (Includes Not Applicable, Unknown)      904
Asian Or Pacific Islander                     715
American Indian                                91
Name: Race, dtype: int64

In [9]:
#91889+73240+1540+715+91

167475

In [10]:
#data_2010_unique = data_2010.drop_duplicates(subset=['person_id'], keep='first')

In [11]:
#data_2010_unique['Race'].value_counts()

White Caucasian (Non-Hispanic)              36534
Black (Non-Hispanic)                        32222
Hispanic                                      770
Other (Includes Not Applicable, Unknown)      510
Asian Or Pacific Islander                     329
American Indian                                56
Name: Race, dtype: int64

In [12]:
#print('2010 Missing Race Data:', 
#      round(((arrests_2010 - (36534+32222+770+329+56)) / arrests_2010)*100, 2), 
#      '%')

2010 Missing Race Data: 1.09 %


# Split to get counties only (matches Census)

In [9]:
# I have the zips but not the counties - will need to convert to counties
data_2010.Address[0:4]

0    KING GEORGE,VA 22485
1    KING GEORGE,VA 22485
2    KING GEORGE,VA 22485
3    KING GEORGE,VA 22485
Name: Address, dtype: object

In [14]:
import addfips
import zipcodes

In [15]:
# test libraries to make sure they work
# this function matches county to zip
af = addfips.AddFIPS()
# test that it works based on 2 zipcodes --> counties I know
info = zipcodes.matching('22901') # my current residence (VA)
print(info[0]['county'])
info = zipcodes.matching('49046') # my hometown (MI)
print(info[0]['county'])

Albemarle County
Barry County


In [16]:
# step 3: we need zipcodes, we we'll need to extract them from the addresses
data_2010[['city_state', 'zip']] = data_2010.Address.str.rsplit(" ", n=1, expand=True,)
data_2010_zips = data_2010[data_2010['zip'].notnull()]

# a few zip codes create an error because they don't have a corresponding county,
# this should be rewritten so that the loop happening a few lines later learns
# to just past over these zips to avoid breaking (and for when we add data later)
data_2010_zips = data_2010_zips[~data_2010_zips['zip'].str.contains('-')]   # there's a messed up zip in row 50k
data_2010_zips = data_2010_zips[data_2010_zips['zip'].str.contains('(?<!\n)[\d]{5,6}[\-]?[\d]*')]
data_2010_zips = data_2010_zips[data_2010_zips['city_state'].str.contains('VA')]

In [17]:
data_2010_zips

,HearingDate,HearingResult,HearingJury,HearingPlea,HearingType,HearingRoom,fips,Filed,Commencedby,Locality,...,RestrictionEndDate,VAAlcoholSafetyAction,RestitutionPaid,RestitutionAmount,Military,TrafficFatality,AppealedDate,person_id,city_state,zip
0,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710,"KING GEORGE,VA",22485
1,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710,"KING GEORGE,VA",22485
2,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710,"KING GEORGE,VA",22485
3,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710,"KING GEORGE,VA",22485
4,2010-12-17,Not Guilty,t,NaN,Jury Trial,NaN,99,2010-09-23,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166070000000710,"KING GEORGE,VA",22485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168806,2010-01-05,Sent,NaN,NaN,Sentencing,NaN,19,2009-01-23,Direct Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253030000000577,"FOREST, VA",24551
168807,2010-01-05,Sent,NaN,NaN,Sentencing,NaN,19,2009-01-23,Direct Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253030000000577,"FOREST, VA",24551
168808,2010-01-05,Nolle Prosequi,NaN,NaN,Plea,NaN,19,2009-07-07,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265101000000248,"FOREST, VA",24551
168809,2010-01-05,Nolle Prosequi,NaN,NaN,Plea,NaN,19,2009-07-07,Indictment,COMMONWEALTH OF VA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265101000000248,"FOREST, VA",24551


In [18]:
data_2010_zips['county'] = 0
data_2010_zips['state']  = 'Virginia'

In [19]:
# add in the county names 
# we'll get in error because we are setting values for entire columns to be 0 (so we can fill them in)
for i in range(len(data_2010_zips)):
    # we can use try-except here because we don't need the bad zip entries to be included - they will just go
    # towards the final "missing" chunk, anyways. 
    try:
        info = zipcodes.matching(list(data_2010_zips['zip'])[i])
    except:
        pass
    if info == []:
        pass
    else: 
        data_2010_zips['county'][i] = info[0]['county']

<ipython-input-19-c97d8f21594d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2010_zips['county'][i] = info[0]['county']
/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [20]:
i
data_2010_zips[i:i+1]

In [21]:
print('2010 Missing Address Data:', 
      round(((arrests_2010 - (36534+32222+770+329+56)) / arrests_2010)*100, 2), 
      '%')

# Run it on all years

In [29]:
file = data_2010_zips
file_zips = file[file['zip'].notnull()]
range(100000, len(file_zips))

range(100000, 128541)

In [16]:
data_2010_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2010.csv')
data_2011_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2011.csv')
data_2012_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2012.csv')
data_2013_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2013.csv')
data_2014_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2014.csv')
data_2015_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2015.csv')
data_2016_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2016.csv')
data_2017_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2017.csv')
data_2018_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2018.csv')
data_2019_zips = pd.read_csv('circuit_data/zips_circuit_criminal_2019.csv')

/Library/Python/3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (40,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Library/Python/3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28,40,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Library/Python/3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (40,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Library/Python/3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (28,40,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [45]:
def get_counties(file):
    """Adds the county of residence for every person with a valid zipcode."""
    
    # step 3: we need zipcodes, we we'll need to extract them from the addresses
    file[['city_state', 'zip']] = file.Address.str.rsplit(" ", n=1, expand=True,)
    file_zips = file[file['zip'].notnull()]
    
    # a few zip codes create an error because they don't have a corresponding county,
    # this should be rewritten so that the loop happening a few lines later learns
    # to just past over these zips to avoid breaking (and for when we add data later)
    file_zips = file_zips[~file_zips['zip'].str.contains('-')]
    file_zips = file_zips[file_zips['zip'].str.contains('(?<!\n)[\d]{5,6}[\-]?[\d]*')]
    file_zips = file_zips[file_zips['city_state'].str.contains('VA')]
    
    # create space for few values
    file_zips['county'] = 0
    file_zips['state']  = 'Virginia'
    
    # add in the county names 
    for i in range(0, len(file_zips)):
        # we can use try-except here because we don't need the bad zip entries to be included - they will just go
        # towards the final "missing" chunk, anyways. 
        if file_zips['county'][i] == 0:
            try:
                info = zipcodes.matching(list(file_zips['zip'])[i])
            except:
                pass
            if info == []:
                pass
            else: 
                file_zips['county'][i] = info[0]['county']
        return file_zips

In [23]:
data_2010_zips = get_counties(data_2010)
data_2011_zips = get_counties(data_2011)
data_2012_zips = get_counties(data_2012)
data_2013_zips = get_counties(data_2013)
data_2014_zips = get_counties(data_2014)
data_2015_zips = get_counties(data_2015)
data_2016_zips = get_counties(data_2016)
data_2017_zips = get_counties(data_2017)
data_2018_zips = get_counties(data_2018)
data_2019_zips = get_counties(data_2019)

<ipython-input-22-892df4656871>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_zips['county'][i] = info[0]['county']
/Users/amawest/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Work out the bugs that seem to be a timeout error

# Sweet! Once we have counties, we can officially compare the counts against the Census data

In [ ]:
# saved zipcodes to csv
data_2010_zips_2.to_csv('circuit_data/zips_circuit_criminal_2010.csv')
data_2011_zips_2.to_csv('circuit_data/zips_circuit_criminal_2011.csv')
data_2012_zips_2.to_csv('circuit_data/zips_circuit_criminal_2012.csv')
data_2013_zips_2.to_csv('circuit_data/zips_circuit_criminal_2013.csv')
data_2014_zips_2.to_csv('circuit_data/zips_circuit_criminal_2014.csv')
data_2015_zips_2.to_csv('circuit_data/zips_circuit_criminal_2015.csv')
data_2016_zips_2.to_csv('circuit_data/zips_circuit_criminal_2016.csv')
data_2017_zips_2.to_csv('circuit_data/zips_circuit_criminal_2017.csv')
data_2018_zips_2.to_csv('circuit_data/zips_circuit_criminal_2018.csv')
data_2019_zips_2.to_csv('circuit_data/zips_circuit_criminal_2019.csv')

In [24]:
data_2010_zips.to_csv('circuit_data/zips_circuit_criminal_2010.csv')
data_2011_zips.to_csv('circuit_data/zips_circuit_criminal_2011.csv')
data_2012_zips.to_csv('circuit_data/zips_circuit_criminal_2012.csv')
data_2013_zips.to_csv('circuit_data/zips_circuit_criminal_2013.csv')
data_2014_zips.to_csv('circuit_data/zips_circuit_criminal_2014.csv')
data_2015_zips.to_csv('circuit_data/zips_circuit_criminal_2015.csv')
data_2016_zips.to_csv('circuit_data/zips_circuit_criminal_2016.csv')
data_2017_zips.to_csv('circuit_data/zips_circuit_criminal_2017.csv')
data_2018_zips.to_csv('circuit_data/zips_circuit_criminal_2018.csv')
data_2019_zips.to_csv('circuit_data/zips_circuit_criminal_2019.csv')

In [46]:
data_2010_zips_2 = get_counties(data_2010_zips)
data_2011_zips_2 = get_counties(data_2011_zips)
data_2012_zips_2 = get_counties(data_2012_zips)
data_2013_zips_2 = get_counties(data_2013_zips)
data_2014_zips_2 = get_counties(data_2014_zips)
data_2015_zips_2 = get_counties(data_2015_zips)
data_2016_zips_2 = get_counties(data_2016_zips)
data_2017_zips_2 = get_counties(data_2017_zips)
data_2018_zips_2 = get_counties(data_2018_zips)
data_2019_zips_2 = get_counties(data_2019_zips)